## Lab 11 - Working with Word Embeddings

### 1. Load Data
If this code cell fails to run, then uncomment the first line and re-run it. This will ensure that tensorflow datasets are installed.

In [ ]:
# !pip install -q tensorflow-datasets
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)



Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete80OQW9/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete80OQW9/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete80OQW9/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### 2. Data Preparation

1. Define lists for the training and testing sentences and labels
2. Iterate over training data, extract sentence and labels, s and l are tensors, so calling numpy() will extract their values
3. Do the same for the test data

In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

# Print the first item in training_sentences and see what the review looks like.
INDEX = 0
review = training_sentences[INDEX].split('.')
for r in review:
  print(r)
#print(training_sentences[INDEX])
print(training_labels[INDEX])


b"This was an absolutely terrible movie
 Don't be lured in by Christopher Walken or Michael Ironside
 Both are great actors, but this must simply be their worst role in history
 Even their great acting could not redeem this movie's ridiculous storyline
 This movie is an early nineties US propaganda piece
 The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions
 Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning
 I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name
 I could barely sit through it
"
0


### 3. Tokenise words using Tokenizer() and pad them using pad_sequences()



> The max length will set to 16



In [ ]:
vocab_size = 1000
embedding_dim = 32
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
print(sequences[0])
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

# Print the word index and see the first few words in it.
print(word_index)



[59, 12, 14, 35, 439, 400, 18, 174, 29, 1, 9, 33, 1, 1, 42, 496, 1, 197, 25, 88, 156, 19, 12, 211, 340, 29, 70, 248, 213, 9, 486, 62, 70, 88, 116, 99, 24, 1, 12, 1, 657, 777, 12, 18, 7, 35, 406, 1, 178, 1, 426, 2, 92, 1, 140, 72, 149, 55, 2, 1, 1, 72, 229, 70, 1, 16, 1, 1, 1, 1, 1, 1, 3, 40, 1, 119, 1, 17, 1, 14, 163, 19, 4, 1, 927, 1, 9, 4, 18, 13, 14, 1, 5, 102, 148, 1, 11, 240, 692, 13, 44, 25, 101, 39, 12, 1, 1, 39, 1, 1, 52, 409, 11, 99, 1, 874, 145, 10]
{'<OOV>': 1, 'the': 2, 'and': 3, 'a': 4, 'of': 5, 'to': 6, 'is': 7, 'br': 8, 'in': 9, 'it': 10, 'i': 11, 'this': 12, 'that': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'movie': 18, 'but': 19, 'film': 20, "'s": 21, 'on': 22, 'you': 23, 'not': 24, 'are': 25, 'his': 26, 'he': 27, 'have': 28, 'be': 29, 'one': 30, 'all': 31, 'at': 32, 'by': 33, 'they': 34, 'an': 35, 'who': 36, 'so': 37, 'from': 38, 'like': 39, 'her': 40, "'t": 41, 'or': 42, 'just': 43, 'there': 44, 'about': 45, 'out': 46, "'": 47, 'has': 48, 'if': 49, 'some': 50, 

### 4. View Effects of Tokenising and Padding



> Rarely used words and often names of things have not made it into the list, since I guess the list only takes the most regular ones. 



In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

index = 2
print(training_sentences[index])
print(decode_review(padded[index]))


b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.'
<OOV> <OOV> the <OOV> <OOV> <OOV> in a superb <OOV> and <OOV> <OOV> and <OOV> <OOV> give enjoyable performances as they always seem to do br br but come on hollywood a <OOV> telling the people of <OOV> city <O

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence]) # tokenizer expects a 'list' of sentences so put our one sentence into a list
print(sequence)

[[11, 68, 105, 12, 7, 491, 1]]


### 5. Neural Network Definition & Compilation


> It's a binary classification with sigmoid activation.




In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 32)           32000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3840)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 23046     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 55,053
Trainable params: 55,053
Non-trainable params: 0
_________________________________________________________________


### 6. Training and Validation

> The model is overfitting after the first epoch. While training accuracy gets extremly high, validation even decreases slightly during the epochs.

In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5251 - accuracy: 0.7250 - val_loss: 0.3893 - val_accuracy: 0.8224
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3733 - accuracy: 0.8370 - val_loss: 0.4054 - val_accuracy: 0.8150
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2821 - accuracy: 0.8864 - val_loss: 0.4570 - val_accuracy: 0.7952
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1772 - accuracy: 0.9412 - val_loss: 0.5533 - val_accuracy: 0.7887
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0946 - accuracy: 0.9748 - val_loss: 0.6923 - val_accuracy: 0.7703
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0449 - accuracy: 0.9914 - val_loss: 0.8467 - val_accuracy: 0.7729
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0191 - accuracy: 0.9973 - val_loss: 0.9696 - val_accuracy: 0.7729
Epoch 

### 7. Converting Embeddings Back to Words




In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


### 8. Writing the Vectors and Metadata into TSV Files

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

### 9. Download the TSV Files

In [ ]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
jovian.commit(project="deeplearning-lab11-embeddings")